In [5]:
from pymongo import MongoClient
from bson import ObjectId
import json
import openai
import os
from dotenv import load_dotenv
import gradio as gr
from qdrant_client import QdrantClient


In [9]:

mongoDBClient = MongoClient("mongodb://localhost:27017/")
db = mongoDBClient['clearml']
collection = db['crawled_data']
client = QdrantClient(host="localhost", port=6333)


In [10]:

load_dotenv()

openai.api_key = os.getenv('OPEN_AI_API_KEY')

def generate_embeddings(text):
    print(text)
    response = openai.embeddings.create(
        input=text,
        model="text-embedding-ada-002"
    )
    return response.data[0].embedding


In [11]:
def RAG_Context (query):
    query_embedding = generate_embeddings(query)
    results = client.search(
    collection_name="robotics-documents",
    query_vector=query_embedding,
    limit=3
    )
    context = "\n\n".join([f"Title: {match.payload['title']}\n{match.payload['content']}" for match in results])
    return context
    

In [14]:
import gradio as gr

# Prefilled suggestions for the dropdown
suggestions = [
    "Tell me how can I navigate to a specific pose - include replanning aspects in your answer.",
    "What is ROS2",
    "What are the three different communication channels?", 
    "Tell me how can I navigate to a specific pose - include replanning aspects in your answer. Can you provide me with code for this task?"
]

# Function to handle the query

from langchain_community.llms import Ollama
import gradio as gr

# Initialize the Ollama model
llm = Ollama(model="https://huggingface.co/CojoFitz/model")

# Define a function to process input with the model
def query_model(input_text):
    try:
        response = llm.invoke(RAG_Context(input_text) + input_text)
        return response
    except Exception as e:
        return f"Error: {str(e)}"
# Function to handle dropdown selection and refill the text box
def update_text(selected):
    return selected  # The selected suggestion replaces the text box content

# Gradio interface setup
with gr.Blocks() as demo:
    with gr.Row():
        dropdown = gr.Dropdown(
            choices=suggestions, 
            label="Suggestions", 
            interactive=True
        )
        textbox = gr.Textbox(
            placeholder="Type your query here...",
            label="Your Query",
            lines=2
        )
    # Update textbox when dropdown changes
    dropdown.change(update_text, inputs=[dropdown], outputs=[textbox])
    submit_button = gr.Button("Submit")
    response_box = gr.Textbox(label="Response")

    # Handle submission
    submit_button.click(query_model, inputs=[textbox], outputs=[response_box])

demo.launch()


* Running on local URL:  http://127.0.0.1:7880

To create a public link, set `share=True` in `launch()`.


What is ROS2
